In [1]:
import pandas as pd
import random 
import os
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

import os
import re

from datasets import load_dataset
import pandas as pd

import requests
import json

from statistics import mean
import numpy as np

import random
import csv
import nltk
from nltk.corpus import gutenberg

import random
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors

In [2]:
def text_rank(text, compression_rate):
    sentences = sent_tokenize(text)
    sort_dict = {}
    for i in range(len(sentences)):
        sort_dict[i] = sentences[i]

    stop_words = set(stopwords.words('english'))
    preprocessed_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [word for word in words if word.isalnum() and word not in stop_words]
        preprocessed_sentences.append(words)
    
    # Load pre-trained Word2Vec model
    w2v_model = KeyedVectors.load_word2vec_format('../models/GoogleNews-vectors-negative300.bin.gz', binary=True)
    
    # Function to calculate the average word embedding for a sentence
    def get_sentence_embedding(sentence):
        embeddings = [w2v_model[word] for word in sentence if word in w2v_model]
        if not embeddings:
            return None
        return np.mean(embeddings, axis=0)
    
    sentence_embeddings = [get_sentence_embedding(sentence) for sentence in preprocessed_sentences]
    sentence_embeddings = np.array(sentence_embeddings)
    
    similarity_matrix = np.dot(sentence_embeddings, sentence_embeddings.T) / (
        np.linalg.norm(sentence_embeddings, axis=1, keepdims=True) * np.linalg.norm(sentence_embeddings, axis=1)
    )
    
    damping_factor = 0.85  
    scores = np.ones(len(sentences))
    prev_scores = np.zeros(len(sentences))
    while np.sum(np.abs(scores - prev_scores)) > 0.001:
        prev_scores = np.copy(scores)
        for i in range(len(sentences)):
            scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[:, i] * scores) / np.sum(similarity_matrix[:, i])

    ranked_sentences = sorted(([scores[i], sentence] for i, sentence in enumerate(sentences)), reverse=True)
    max_words = len(word_tokenize(text)) * (1-compression_rate)
    words=0
    chosen_texts = []
    for text in ranked_sentences:
        if words<=max_words:
            sen_length = len(word_tokenize(text[1])) # https://www.guru99.com/tokenize-words-sentences-nltk.html
            words += sen_length
            chosen_texts.append(text[1])
    chosen_dict = {}
    for i in range(len(sort_dict)):
        for text in chosen_texts:
            if sort_dict[i] == text:
                chosen_dict[i] = text
    summary_sentences = chosen_dict.values()
    summary = ' '.join(summary_sentences)
    return summary


In [3]:
print(text_rank('''The World Cup co-host looked to have earned itself a route back into the game midway through the second half when Jacqui Hand’s looping header floated over a despairing Olivia McDaniel in goal, but it was later ruled out by the video assistant referee (VAR) for offside.

The World Cup debutant was able to withstand New Zealand pressure, including a truly remarkable diving save from McDaniel in added time at the end of the game, to earn a historic victory, sparking scenes of jubilant celebrations.

“I literally can’t put it into words,” Bolden said afterwards. “This has been a dream of mine as a little kid to just be here at the World Cup, let alone even score.

“I couldn’t have done it without my teammates, the staff, the fans, the Philippines as a whole. It’s just amazing right now to feel this win and this energy in this stadium right now, so it’s just amazing.”''', 0.5))

/var/folders/c5/2zgnc0p131qddyd091x_x8w00000gn/T/ipykernel_19314/2356465635.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence_embeddings = np.array(sentence_embeddings)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'

In [64]:
df = pd.read_csv("../data/Result/reference_summaries_dataset.csv")
df.head()

,Text,Class,Summary
0,I have heard nothing from the Ambassador about...,Political speech,I have heard nothing from the Ambassador about...
1,I think it is in the public interest to procee...,Political speech,I think it is in the public interest to procee...
2,The A-11 aircraft now at Edwards Air force Bas...,Political speech,"for example, one of the most important technic..."
3,It is one of the most comprehensive bills in t...,Political speech,It is one of the most comprehensive bills in t...
4,"So long as there remains a man without a job, ...",Political speech,No American conscience can be at peace while a...


# Evaluierung

In [65]:
from rouge import Rouge
import pandas as pd

def evaluate_rouge_scores(df):
    rouge = Rouge()

    rouge_scores = {
        'all_classes': {
            'rouge-1': {
                'f': [],
                'p': [],
                'r': []
            }
        }
    }

    # Get unique classes
    classes = df['Class'].unique()

    for cls in classes:
        rouge_scores[cls] = {
            'rouge-1': {
                'f': [],
                'p': [],
                'r': []
            }
        }

    for _, row in df.iterrows():
        text = row['Text']
        reference_summary = row['Summary']
        cls = row['Class']


        # Generate summary with compression rate
        compression_rate = len(reference_summary.split()) / len(text.split())
        summary = text_rank(text, compression_rate=compression_rate)

        # calculate rouge 1
        rouge_scores_all = rouge.get_scores(summary, reference_summary)[0]
        rouge_scores_cls = rouge_scores[cls]

        # Add rouge score values
        rouge_scores_all_cls = rouge_scores_cls['rouge-1']
        rouge_scores_all_cls['f'].append(rouge_scores_all['rouge-1']['f'])
        rouge_scores_all_cls['p'].append(rouge_scores_all['rouge-1']['p'])
        rouge_scores_all_cls['r'].append(rouge_scores_all['rouge-1']['r'])

        # Add rouge score values for all classes
        rouge_scores_all_all = rouge_scores['all_classes']['rouge-1']
        rouge_scores_all_all['f'].append(rouge_scores_all['rouge-1']['f'])
        rouge_scores_all_all['p'].append(rouge_scores_all['rouge-1']['p'])
        rouge_scores_all_all['r'].append(rouge_scores_all['rouge-1']['r'])

    return rouge_scores

In [66]:
# make sure all strings
df['Text'] = df['Text'].astype(str)
df['Summary'] = df['Summary'].astype(str)

In [69]:
rouge_scores = evaluate_rouge_scores(df)

# calculate ROUGE-1-Scores for all classes
print("ROUGE-1 Scores für alle Klassen:")
print(rouge_scores['all_classes']['rouge-1'])

# calculate ROUGE-1-Scores for each class
for cls, scores in rouge_scores.items():
    if cls != 'all_classes':
        print(f"ROUGE-1 Scores für Klasse {cls}:")
        print(scores['rouge-1'])

/var/folders/c5/2zgnc0p131qddyd091x_x8w00000gn/T/ipykernel_46965/2356465635.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence_embeddings = np.array(sentence_embeddings)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'

In [62]:
# calculate avg, max, min for all classes
print("ROUGE-1 Scores für alle Klassen:")
all_classes_scores = rouge_scores['all_classes']['rouge-1']
all_classes_f1_avg = sum(all_classes_scores['f']) / len(all_classes_scores['f'])
all_classes_p_avg = sum(all_classes_scores['p']) / len(all_classes_scores['p'])
all_classes_r_avg = sum(all_classes_scores['r']) / len(all_classes_scores['r'])
all_classes_f1_min = min(all_classes_scores['f'])
all_classes_p_min = min(all_classes_scores['p'])
all_classes_r_min = min(all_classes_scores['r'])
all_classes_f1_max = max(all_classes_scores['f'])
all_classes_p_max = max(all_classes_scores['p'])
all_classes_r_max = max(all_classes_scores['r'])
print(f"F1-Score Durchschnitt für alle Klassen: {all_classes_f1_avg:.2f} (Min: {all_classes_f1_min:.2f}, Max: {all_classes_f1_max:.2f})")
print(f"Precision Durchschnitt für alle Klassen: {all_classes_p_avg:.2f} (Min: {all_classes_p_min:.2f}, Max: {all_classes_p_max:.2f})")
print(f"Recall Durchschnitt für alle Klassen: {all_classes_r_avg:.2f} (Min: {all_classes_r_min:.2f}, Max: {all_classes_r_max:.2f})")

# calculate avg, max, min for each class
for cls, scores in rouge_scores.items():
    if cls != 'all_classes':
        print(f"ROUGE-1 Scores für Klasse {cls}:")
        class_scores = scores['rouge-1']
        class_f1_avg = sum(class_scores['f']) / len(class_scores['f'])
        class_p_avg = sum(class_scores['p']) / len(class_scores['p'])
        class_r_avg = sum(class_scores['r']) / len(class_scores['r'])
        class_f1_min = min(class_scores['f'])
        class_p_min = min(class_scores['p'])
        class_r_min = min(class_scores['r'])
        class_f1_max = max(class_scores['f'])
        class_p_max = max(class_scores['p'])
        class_r_max = max(class_scores['r'])
        print(f"F1-Score Durchschnitt für Klasse {cls}: {class_f1_avg:.2f} (Min: {class_f1_min:.2f}, Max: {class_f1_max:.2f})")
        print(f"Precision Durchschnitt für Klasse {cls}: {class_p_avg:.2f} (Min: {class_p_min:.2f}, Max: {class_p_max:.2f})")
        print(f"Recall Durchschnitt für Klasse {cls}: {class_r_avg:.2f} (Min: {class_r_min:.2f}, Max: {class_r_max:.2f})")

ROUGE-1 Scores für alle Klassen:
F1-Score Durchschnitt für alle Klassen: 0.51 (Min: 0.00, Max: 1.00)
Precision Durchschnitt für alle Klassen: 0.67 (Min: 0.00, Max: 1.00)
Recall Durchschnitt für alle Klassen: 0.55 (Min: 0.00, Max: 1.00)
ROUGE-1 Scores für Klasse Political speech:
F1-Score Durchschnitt für Klasse Political speech: 0.52 (Min: 0.00, Max: 0.99)
Precision Durchschnitt für Klasse Political speech: 0.67 (Min: 0.00, Max: 1.00)
Recall Durchschnitt für Klasse Political speech: 0.55 (Min: 0.00, Max: 1.00)
ROUGE-1 Scores für Klasse News:
F1-Score Durchschnitt für Klasse News: 0.51 (Min: 0.12, Max: 0.87)
Precision Durchschnitt für Klasse News: 0.64 (Min: 0.16, Max: 1.00)
Recall Durchschnitt für Klasse News: 0.56 (Min: 0.07, Max: 1.00)
ROUGE-1 Scores für Klasse Jurisdiction:
F1-Score Durchschnitt für Klasse Jurisdiction: 0.55 (Min: 0.14, Max: 0.98)
Precision Durchschnitt für Klasse Jurisdiction: 0.68 (Min: 0.08, Max: 1.00)
Recall Durchschnitt für Klasse Jurisdiction: 0.59 (Min: 0.10,